In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

In [2]:
csv1 = "movie_data/movies.csv"
csv2 = "movie_data/ratings.csv"
csv3 = "movie_data/genome-scores.csv"
csv4 = "movie_data/genome-tags.csv"
csv5 = "movie_data/links.csv"
csv6 = "movie_data/tags.csv"

In [3]:
movies_df = pd.read_csv(csv1)
ratings_df = pd.read_csv(csv2)
genome_scores_df = pd.read_csv(csv3)
genome_tags_df = pd.read_csv(csv4)
links_df = pd.read_csv(csv5)
tags_df = pd.read_csv(csv6)

In [ ]:
print(f'{len(movies_df)} movies')
print(f'{len(ratings_df)} ratings')

In [ ]:
movies_df.head()

In [ ]:
ratings_df.head()

In [ ]:
# # Number of ratings for each userId
# ratings_df.groupby(ratings_df['userId'])['rating'].count()

In [ ]:
genome_scores_df.head()

In [4]:
relevant_genome_scores_df = genome_scores_df.loc[genome_scores_df['relevance'] > .85]

In [ ]:
genome_tags_df.head()

In [ ]:
links_df.head()

In [ ]:
tags_df.head()

In [ ]:
new_df = ratings_df.merge(movies_df, how="inner")
new_df = new_df.drop(columns=['title', 'genres', 'timestamp'])
new_df.head()

In [ ]:
new_df["rating"] = np.where(new_df["rating"] >= 3.5, 1, 0)

In [ ]:
new_df

In [ ]:
X = new_df.drop('rating', axis=1)
y = new_df['rating']

In [ ]:
from sklearn import preprocessing
from sklearn import utils

#convert y values to categorical values
lab = preprocessing.LabelEncoder()
y_transformed = lab.fit_transform(y)

#view transformed values
print(y_transformed)
X_train, X_test, y_train, y_test = train_test_split(X, y_transformed, random_state=1)

In [ ]:
classifier = LogisticRegression()
classifier.fit(X_train, y_train)
print(f"Training Data Score: {classifier.score(X_train, y_train)}")
print(f"Testing Data Score: {classifier.score(X_test, y_test)}")

In [ ]:
# movies_df.head()

In [ ]:
# movie_ratings = movies_df.merge(ratings_df, how="inner")
# movie_ratings.head()

In [ ]:
# ratings_genome = movie_ratings.merge(relevant_genome_scores_df, how="inner")
# ratings_genome.head()

In [5]:
movie_ratings_genome = ratings_df.merge(relevant_genome_scores_df, how="inner")
movie_ratings_genome.head()

,userId,movieId,rating,timestamp,tagId,relevance
0,1,296,5.0,1147880044,146,0.96200
1,1,296,5.0,1147880044,169,0.90625
2,1,296,5.0,1147880044,240,0.86375
3,1,296,5.0,1147880044,242,0.87350
4,1,296,5.0,1147880044,255,0.92250


In [6]:
movie_ratings_genome["rating"] = np.where(movie_ratings_genome["rating"] >= 3.5, 1, 0)

In [7]:
movie_ratings_genome.head()

,userId,movieId,rating,timestamp,tagId,relevance
0,1,296,1,1147880044,146,0.96200
1,1,296,1,1147880044,169,0.90625
2,1,296,1,1147880044,240,0.86375
3,1,296,1,1147880044,242,0.87350
4,1,296,1,1147880044,255,0.92250


In [ ]:
X = movie_ratings_genome.drop('rating', axis=1)
y = movie_ratings_genome['rating']

In [ ]:
def genre_ratings(genre):
    df = movies_df[movies_df['genres'].str.contains(genre, regex=False)]
    genre_df = ratings_df.merge(df, how="inner")
    return genre_df

In [ ]:
genre_ratings("Comedy")